In [2]:
import os
import re
import torch
import random
# from train_data import input_data
# import argparse
import numpy as np
from collections import defaultdict

In [2]:
data_path = "../aminer/"
neigh_dict = {}

In [11]:
# # Neighbour reader for all nodes
# data_path = "../aminer/"
# neigh_dict = defaultdict(list)
# file_names = ['a_p_train', 'p_a_train', 'p_v_train', 'v_p_train']

# for k in range(len(file_names)):
    
#     with open(data_path + file_names[k] + '.txt', 'r') as file:            
#         lines = file.readlines()
    
#     for i, line in enumerate(lines):
#         node_id, neigh = line.strip().split(':')
#         neigh_list = neigh.split(',')
#         # print(node_id, neigh_list)
        
#         if k == 0:
#             for node in neigh_list:
#                 neigh_dict[f'a{node_id}'].append(f'p{node}')
#         elif k == 1:
#             for node in neigh_list:
#                 neigh_dict[f'p{node_id}'].append(f'a{node}')
#         elif k == 2:
#             for node in neigh_list:
#                 neigh_dict[f'p{node_id}'].append(f'v{node}')
#         else: 
#             for node in neigh_list:
#                 if int(node) < 13250:
#                     neigh_dict[f'v{node_id}'].append(f'p{node}')

In [12]:
# print(list(neigh_dict.keys())[:100])

['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37', 'a38', 'a39', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46', 'a47', 'a48', 'a49', 'a50', 'a51', 'a52', 'a53', 'a54', 'a55', 'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63', 'a64', 'a65', 'a66', 'a67', 'a68', 'a69', 'a70', 'a71', 'a72', 'a73', 'a74', 'a75', 'a76', 'a77', 'a78', 'a79', 'a80', 'a81', 'a82', 'a83', 'a84', 'a85', 'a86', 'a87', 'a88', 'a89', 'a90', 'a91', 'a92', 'a93', 'a94', 'a95', 'a96', 'a97', 'a98', 'a99']


In [ ]:
# random_walks = defaultdict(list)

# for node in list(neigh_dict.keys()):
#     print(node)
#     curNode = node
    
#     walk_size, a_size, p_size, v_size = 0,0,0,0
    
#     while walk_size < 100:
#         prob =  random.random()
#         if prob < 0.5:
#             curNode = node
#         else:
#             if curNode not in list(neigh_dict.keys()):
#                 print(node, curNode)
#             curNode = random.choice(neigh_dict[curNode])
#             if curNode != node:
#                 if curNode[0] == 'a' and a_size < 46:
#                     random_walks[node].append(curNode)
#                     a_size += 1
#                 elif curNode[0] == 'p' and p_size < 46:
#                     random_walks[node].append(curNode)
#                     p_size += 1
#                 elif curNode[0] == 'v' and v_size < 11:
#                     random_walks[node].append(curNode)
#                     v_size += 1
                
#                 walk_size += 1

In [14]:
# print(list(random_walks.keys())[:100])

['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37', 'a38', 'a39', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46', 'a47', 'a48', 'a49', 'a50', 'a51', 'a52', 'a53', 'a54', 'a55', 'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63', 'a64', 'a65', 'a66', 'a67', 'a68', 'a69', 'a70', 'a71', 'a72', 'a73', 'a74', 'a75', 'a76', 'a77', 'a78', 'a79', 'a80', 'a81', 'a82', 'a83', 'a84', 'a85', 'a86', 'a87', 'a88', 'a89', 'a90', 'a91', 'a92', 'a93', 'a94', 'a95', 'a96', 'a97', 'a98', 'a99']


In [17]:
# with open('het_neigh_train.txt', 'w') as file:
#     for node in list(random_walks.keys()):
#         file.write(node + ':' + ','.join(random_walks[node])+'\n')

In [19]:
# node net embedding generator
from gensim.models import Word2Vec
from itertools import *
dimen = 128
window = 5

def read_random_walk_corpus():
    walks, node_ids = [], []
    with open("het_neigh_train.txt", 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            parts = line.strip().split(':')
            node_ids.append(parts[0])
            path = parts[1].split(',')
            walks.append(path)

    return walks, node_ids

walk_corpus, node_ids = read_random_walk_corpus()

# Train the Word2Vec model
model = Word2Vec(walk_corpus, vector_size=dimen, window=window, min_count=0, workers=8, sg=1, hs=0, negative=5)

# Save node embeddings to a text file
with open("node_net_embedding.txt", 'w') as file:
    for i, node_id in enumerate(node_ids):
        embedding = model.wv[node_id]
        embedding_str = " ".join(str(val) for val in embedding)
        file.write(node_id + " " + embedding_str + "\n")
model.wv.save_word2vec_format("node_net_embedding.txt")

In [23]:
# with open(data_path + "p_p_train.txt", 'r') as file1:            
#     lines1 = file1.readlines()
    
# with open(data_path + "p_p_edge_pred.txt", 'w') as file2:            

#     for i, line in enumerate(lines1):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         node_neigh= re.split(',', re.split(':', line)[1])
    
#         for neigh in node_neigh:
#             file2.write(node_id+ ',' + neigh + ",1\n")
        
#         for i in range(len(node_neigh)):
#             while True:
#                 rand = str(np.random.randint(0, 13249))
#                 if rand not in node_neigh: break 
#             file2.write(node_id+ ',' + rand + ",0\n")

# for i, line in enumerate(lines2):
#     line = line.strip()
#     node_id = re.split(',', line)[0].strip()
#     id2.append(node_id)
    

In [16]:
# # Check if list2 is a subset of list1
# is_subset = set(id2).issubset(set(id1))

# # Find elements from list2 that are not in list1
# elements_not_in_list1 = [item for item in id2 if item not in id1]

# # Print the results
# print(f"Is list2 a subset of list1? {is_subset}")
# print(f"Elements from list2 not in list1: {elements_not_in_list1}")

Is list2 a subset of list1? True
Elements from list2 not in list1: []


In [ ]:

# with open(data_path + "het_neigh_train.txt", 'r') as filex:
#     lines3 = filex.readlines()

# with open("het_neigh_train.txt", 'w') as file5:
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         neigh_list = re.split(',', re.split(':', line)[1])
#         node_type = node_id[0]
#         if i == 0: print(len(neigh_list))
        
#         if node_type == 'v':
#             file5.write(str(node_id)+':')
#         elif node_type == 'a':
#             try:
#                 file5.write('a' + str(dict_a[node_id[1:]]) + ':')
#             except KeyError:
#                 continue
#         else:
#             try:
#                 file5.write('p' + str(dict_p[node_id[1:]]) + ':')
#             except KeyError:
#                 continue
                   
#         for j in range(len(neigh_list)):
#             node_t = neigh_list[j][0]
#             if node_t == 'v':
#                 file5.write(str(neigh_list[j]))
#             elif node_t == 'a':
#                 try:
#                     file5.write('a' + str(dict_a[neigh_list[j][1:]]))
#                 except KeyError:
#                     continue
#             else:
#                 try:
#                     file5.write('p' + str(dict_p[neigh_list[j][1:]]))
#                 except KeyError:
#                     continue
                
#             if j != len(neigh_list) - 1:
#                 file5.write(',')
        
#         # Move to the next line
#         file5.write('\n')

In [10]:
# with open("a_p_train.txt", 'w') as file3:            
#     for i, line in enumerate(lines1):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         neigh_list = re.split(',', re.split(':', line)[1])
#         file3.write(str(dict_a[node_id]) + ':')
                
#         for j in range(len(neigh_list)):
#             file3.write(str(dict_p[neigh_list[j]]))
#             if j != len(neigh_list) - 1:
#                 file3.write(',')
        
#         # Move to the next line
#         file3.write('\n')

In [12]:
# with open("p_a_train.txt", 'w') as file4:            
#     for i, line in enumerate(lines2):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         neigh_list = re.split(',', re.split(':', line)[1])
#         file4.write(str(dict_p[node_id]) + ':')
                
#         for j in range(len(neigh_list)):
#             file4.write(str(dict_a[neigh_list[j]]))
#             if j != len(neigh_list) - 1:
#                 file4.write(',')
        
#         # Move to the next line
#         file4.write('\n')

In [14]:
# with open(data_path + "p_p_cite_list_train.txt", 'r') as filex:            
#     lines3 = filex.readlines()
    
# with open("p_p_train.txt", 'w') as file5:            
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         neigh_list = re.split(',', re.split(':', line)[1])
#         file5.write(str(dict_p[node_id]) + ':')
                
#         for j in range(len(neigh_list)):
#             file5.write(str(dict_p[neigh_list[j]]))
#             if j != len(neigh_list) - 1:
#                 file5.write(',')
        
#         # Move to the next line
#         file5.write('\n')

In [15]:
# with open(data_path + "v_p_list_train.txt", 'r') as filex:            
#     lines3 = filex.readlines()
    
# with open("v_p_train.txt", 'w') as file5:            
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         neigh_list = re.split(',', re.split(':', line)[1])
#         file5.write(str(node_id) + ':')
                
#         for j in range(len(neigh_list)):
#             file5.write(str(dict_p[neigh_list[j]]))
#             if j != len(neigh_list) - 1:
#                 file5.write(',')
        
#         # Move to the next line
#         file5.write('\n')

In [16]:
# with open(data_path + "p_v_list.txt", 'r') as filex:
#     lines3 = filex.readlines()

# with open("p_v_train.txt", 'w') as file5:
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(':', line)[0]
#         neigh_list = re.split(',', re.split(':', line)[1])
        
#         try:
#             file5.write(str(dict_p[node_id]) + ':')
#         except KeyError:
#             # Handle the case where node_id is not found in dict_p
#             continue
                
#         for j in range(len(neigh_list)):
#             file5.write(str(neigh_list[j]))
#             if j != len(neigh_list) - 1:
#                 file5.write(',')
        
#         # Move to the next line
#         file5.write('\n')

In [17]:
# with open(data_path + "p_title_embed.txt", 'r') as filex:
#     lines3 = filex.readlines()[1:]

# with open("p_title_train.txt", 'w') as file5:
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(' ', line)[0]
#         neigh_list = re.split(' ', line)[1:]
#         if i == 0: print(len(neigh_list))
        
#         try:
#             file5.write(str(dict_p[node_id]) + ' ')
#         except KeyError:
#             # Handle the case where node_id is not found in dict_p
#             continue
                
#         for j in range(len(neigh_list)):
#             file5.write(str(neigh_list[j]))
#             if j != len(neigh_list) - 1:
#                 file5.write(' ')
        
#         # Move to the next line
#         file5.write('\n')

128


In [18]:
# with open(data_path + "p_abstract_embed.txt", 'r') as filex:
#     lines3 = filex.readlines()[1:]

# with open("p_abstract_train.txt", 'w') as file5:
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(' ', line)[0]
#         neigh_list = re.split(' ', line)[1:]
#         if i == 0: print(len(neigh_list))
        
#         try:
#             file5.write(str(dict_p[node_id]) + ' ')
#         except KeyError:
#             # Handle the case where node_id is not found in dict_p
#             continue
                
#         for j in range(len(neigh_list)):
#             file5.write(str(neigh_list[j]))
#             if j != len(neigh_list) - 1:
#                 file5.write(' ')
        
#         # Move to the next line
#         file5.write('\n')

128


In [20]:
# with open(data_path + "node_net_embedding.txt", 'r') as filex:
#     lines3 = filex.readlines()[2:]

# with open("node_net_train.txt", 'w') as file5:
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(' ', line)[0]
#         neigh_list = re.split(' ', line)[1:]
#         node_type = node_id[0]
        
#         if node_type == 'v':
#             file5.write(str(node_id)+' ')
#         elif node_type == 'a':
#             try:
#                 file5.write('a' + str(dict_a[node_id[1:]]) + ' ')
#             except KeyError:
#                 continue
#         else:
#             try:
#                 file5.write('p' + str(dict_p[node_id[1:]]) + ' ')
#             except KeyError:
#                 continue
                   
#         for j in range(len(neigh_list)):
#             file5.write(str(neigh_list[j]))
#             if j != len(neigh_list) - 1:
#                 file5.write(' ')
        
#         # Move to the next line
#         file5.write('\n')

In [21]:
# with open(data_path + "a_class_train.txt", 'r') as filex:
#     lines3 = filex.readlines()

# with open("a_class_train.txt", 'w') as file5:
#     for i, line in enumerate(lines3):
#         line = line.strip()
#         node_id = re.split(',', line)
        
#         try:
#             file5.write(str(dict_a[node_id[0]]) + ',' + str(node_id[1]))
#         except KeyError:
#             continue
                
#         file5.write('\n')